In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from settings import MONSTER_API_KEY
from openai import OpenAI
import time

# --- SETUP ---
SIMILARITY_THRESHOLD = 0.4
MAX_RESULTS = 10
MIN_RESULTS = 1

MONSTER_API_KEY = MONSTER_API_KEY
llm_name = "Meta-Llama"
monster_ai_model_name = {
    "Google-Gemma": "google/gemma-2-9b-it",
    "Mistral": "mistralai/Mistral-7B-Instruct-v0.2",
    "Microsoft-Phi": "microsoft/Phi-3-mini-4k-instruct",
    "Meta-Llama": "meta-llama/Meta-Llama-3.1-8B-Instruct",
}

# --- LOAD DATA ---
df = pd.read_csv("/Users/parakhchaudhary/SHL_Recommendation_System/dataset/dataset_final.csv")
embeddings = np.load("/Users/parakhchaudhary/SHL_Recommendation_System/dataset/dataset_final.npy")



# --- MONSTER API ---
def setup_monster_api():
    return OpenAI(
        base_url="https://llm.monsterapi.ai/v1/",
        api_key=(MONSTER_API_KEY),
    )

def call_monster_api(user_input, context, client, retries=3, wait_time=5):
    attempt = 0
    while attempt < retries:
        try:
            response = client.chat.completions.create(
                model=monster_ai_model_name[llm_name],
                messages=[
                    {"role": "system", "content": context},
                    {"role": "user", "content": user_input},
                ]
            )
            return response.choices[0].message.content.strip()
        except Exception as e:
            if "429" in str(e):
                print(f"Rate limit hit. Retrying in {wait_time}s...")
                time.sleep(wait_time)
                attempt += 1
            else:
                print(f"Monster API error: {e}")
                break
    return "I'm sorry, I couldn't generate a recommendation."

# --- MAIN LOGIC ---
model = SentenceTransformer("all-MiniLM-L6-v2")


def recommend_tests(user_query):
    query_embedding = model.encode([user_query])
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    # We find the similarity

    matching_indices = np.where(similarities >= SIMILARITY_THRESHOLD)[0]
    if len(matching_indices) == 0:
        matching_indices = np.array([np.argmax(similarities)])

    sorted_indices = matching_indices[np.argsort(similarities[matching_indices])[::-1]]
    top_indices = sorted_indices[:MAX_RESULTS]

    results = df.iloc[top_indices].copy()
    results["similarity"] = similarities[top_indices]

    print("Top similarity scores:", sorted(similarities, reverse=True)[:5])
    return results


def generate_llm_response(user_query, recommendations_df, client):
    context = "You are an assessment recommendation assistant for SHL. Your job is to explain clearly which assessments suit the user's requirement."

    top_items = []
    for _, row in recommendations_df.iterrows():
        time = f", {row['Assessment Length']}" if pd.notnull(row['Assessment Length']) else ""
        top_items.append(f"- {row['name']} ({row['test_type']}{time})")

    message = f"""
User Query: \"{user_query}\"

Based on SHL's test catalog, here are the most relevant assessments:

{chr(10).join(top_items)}

Please summarize and explain this recommendation in 3-4 sentences, addressing the user's needs.
"""

    return call_monster_api(user_input=message, context=context, client=client)


# --- DRIVER ---
if __name__ == "__main__":
    query = "ICICI Bank Assistant Admin, Experience required 0-2 years, test should be 30-40 mins long"
    results = recommend_tests(query)

    monster_client = setup_monster_api()
    response = generate_llm_response(query, results.head(5), monster_client)

    print("\n✅ LLM-Generated Recommendation:")
    print(response)


INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
/Users/parakhchaudhary/opt/anaconda3/envs/chaenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
Batches: 100%|██████████| 1/1 [00:00<00:00, 10.31it/s]


Top similarity scores: [0.5914551, 0.547591, 0.53832364, 0.48696387, 0.4781316]


ValueError: invalid literal for int() with base 10: 'Approximate Completion Time in minutes = 9'